In [1]:
import numpy as np
import sys

class NeuralNetMLP(object):

    def __init__(self, n_hidden=30,
                 l2=0., epochs=100, eta=0.001,
                 shuffle=True, minibatch_size=1, seed=None):
        
        if not seed is None:
            np.random.seed(seed)  # 乱数の固定
        self.random = np.random.RandomState(seed)
        self.n_hidden = n_hidden
        self.l2 = l2
        self.epochs = epochs
        self.eta = eta
        self.shuffle = shuffle
        self.minibatch_size = minibatch_size
        
        n_features = 4 #次元
        n_output = 4  # 2次元運動の時 # 出力の数

        ##########################
        ###### 重みの初期化 ######
        ##########################

        # 入力層→隠れ層の重み
        self.b_h = np.zeros((self.n_hidden, self.n_hidden))
        self.w_h = self.random.normal(loc=0.0, scale=0.1,size=(n_features, self.n_hidden))
        
        # 隠れ層の参照ベクトル # 初期化
        self.nodes = np.random.rand(self.n_hidden, self.n_hidden, n_features)  # 0.0~1.0の値で一様分布 # n_features：次元
        
        for x in range (self.n_hidden):
            for y in range(self.n_hidden):
                for z in range(n_features):
                    "今までと結果変わる"
                    #M = X_train.max() 
                    #m = X_train.min()
                    #参照ベクトルを -0.5~0.5(max-min) の一様乱数とする
                    self.nodes[x, y, z] = 0.5 *100* self.nodes[x, y, z]
                    #self.nodes[x, y, z] = 0.5 *(M - m) * self.nodes[x, y, z] + m            
        
        # 隠れ層→出力層の重み
        self.b_out = np.zeros(n_output)
        self.w_out = self.random.normal(loc=0.0, scale=0.1,size=(self.n_hidden, self.n_hidden, n_output))
        
        
    
    def _onehot(self, y, n_classes):
        
        onehot = np.zeros((n_classes, y.shape[0]))
        for idx, val in enumerate(y.astype(int)): #int型に変換
            onehot[val, idx] = 1.
        return onehot.T

    def _sigmoid(self, z):
        return 1. / (1. + np.exp(-np.clip(z, -250, 250)))
    
    
    ####################################################################
    ############################ SOM ###################################
    ###################################################################
     
    def _best_matching_unit(self, X):
        n_features = X.shape[1]
      
        norms = np.zeros((self.n_hidden, self.n_hidden))  # n_hidden×n_hiddenの２次元配列を生成
        for x in range (self.n_hidden):
            for y in range(self.n_hidden):
                for z in range(n_features):
                    norms[x, y] += (self.nodes[x,y,z] - X[z])**2

        bmu_1 = np.argmin(norms)
        bmu = np.unravel_index(bmu_1,(self.n_hidden, self.n_hidden))

        return bmu 

    def _neighbourhood(self, t):
        halflife = float(n_teacher/4)
        initial = float(self.n_hidden /4)
        return initial*np.exp(-t/halflife)

    def _learning_ratio(self, t):
        halflife = float(n_teacher/4)
        initial = 0.1
        return initial*np.exp(-t/halflife)

    def _learning_radius(self, t, d):
        s = self._neighbourhood(t)
        return np.exp(-d**2/(2*s**2)) 
    

    def _forward(self, X):
        """フォワードプロパゲーションのステップを計算"""

        # step 1: 隠れ層の入力     
        z_h = np.dot(X, self.w_h) + self.b_h

        # step 2: 隠れ層の活性化関数 #a_h = self._sigmoid(z_h)
        
        n_teacher = X.shape[0]
        bmu = self._best_matching_unit(X)
        for x in range(self.n_hidden):
            for y in range(self.n_hidden):
                c = np.array([x,y])
                d = np.linalg.norm(c - bmu)
                L = self._learning_ratio(i)
                S = self._learning_radius(i, d)
                for z in range(n_features):
                    self.nodes[x, y, z] += L * S * (X[z] - self.nodes[x, y, z])
        
        a_h = self.nodes
        
        # step 3: 出力層の総入力
        z_out = np.dot(a_h, self.w_out) + self.b_out

        # step 4: 出力層の活性化関数
        a_out = self._sigmoid(z_out)

        return z_h, a_h, z_out, a_out 

    def _compute_cost(self, y_enc, output):
        
        L2_term = (self.l2 *
                   (np.sum(self.w_h ** 2.) +
                    np.sum(self.w_out ** 2.)))
        
        ####ロジスティック関数のコスト関数#####
        ######対数尤度関数#自然対数を最小化するほうが簡単#######
        term1 = -y_enc * (np.log(output))
        term2 = (1. - y_enc) * np.log(1. - output)
        cost = np.sum(term1 - term2) + L2_term
              
        return cost
    
    ######## クラスラベル予測 ###########
    def predict(self, X):
        
        z_h, a_h, z_out, a_out = self._forward(X)
        y_pred = np.argmax(z_out, axis=1)  # 出力層で一番大きい値をとるものを予測に適応
        return y_pred
    
    ######### 重みの学習 ##########
    def fit(self, X_train, y_train_enc):
        
        # BMUとクラスを記録するためのcsvファイルを開く
        bmu_list = np.array([])
        classnum_list =  np.array([])
        
        #ax = plt.subplot()
        
        # エポック数だけトレーニングを繰り返す
        for i in range(self.epochs):

            ##################################
             ### フォワードプロパゲーション###
            #################################

            z_h, a_h, z_out, a_out = self._forward(X_train[i])
                
            bmu = best_matching_unit(X_train)
            y = y_train
                
            bmu_list = np.array(bmu)
            classnum_list = np.array(y)
                
            #df2 = pd.read_csv('bmu.csv')
            #if y[i] != df2['classnum'].any:
                #if y[i]==[0]:#後ろ
                    #col='red'
                    #mark = 'o'

                #elif y[i]==[1]:#前
                    #col='blue'
                    #mark = '^'

            #scale = 1 * df2['bmu'].value_counts()
            #ax.scatter(bmu[0],bmu[1],color = col, marker=mark, s=scale, alpha = 1.0)

            ##############################
            ### バックプロパゲーション###
            #############################
            "＋参照ベクトル修正"
                
            ####誤差行列(コスト関数の微分)####
            # [n_samples, n_classlabels]
            sigma_out = a_out - y_train_enc[i]
                
            "rbf関数"
            ####シグモイド関数の微分#####
            # [n_classlabels, n_hidden]
            sigmoid_derivative_h = a_h * (1. - a_h)
                
            ####誤差行列(コスト関数の入力z微分)#####
            # [n_samples, n_classlabels] dot [n_classlabels, n_hidden]
            # -> [n_samples, n_hidden]
            sigma_h = (np.dot(sigma_out, self.w_out.T) * sigmoid_derivative_h)
                
            #####偏微分係数(コスト関数の重みw微分)##########
            # [n_features, n_samples] dot [n_samples, n_hidden]
            # -> [n_features, n_hidden]
            grad_w_h = np.dot(X_train[i].T, sigma_h)
            grad_b_h = np.sum(sigma_h, axis=0)

            #####偏微分係数(コスト関数の重みw微分)##########
            # [n_hidden, n_samples] dot [n_samples, n_classlabels]
            # -> [n_hidden, n_classlabels]
            grad_w_out = np.dot(a_h.T, sigma_out)
            grad_b_out = np.sum(sigma_out, axis=0)

            # 正則化
            delta_w_h = (grad_w_h + self.l2*self.w_h)
            delta_b_h = grad_b_h # バイアスは正則化しない
                
            #####接続重み修正########勾配に対して反対方向
            self.w_h -= self.eta * delta_w_h
            #######バイアス修正######
            self.b_h -= self.eta * delta_b_h

            # 正則化
            delta_w_out = (grad_w_out + self.l2*self.w_out)
            delta_b_out = grad_b_out  #バイアスは正則化しない
                
            #####接続重み修正########
            self.w_out -= self.eta * delta_w_out
            #######バイアス修正######
            self.b_out -= self.eta * delta_b_out

        return self

nn = NeuralNetMLP(n_hidden = 15, #隠れユニットの行列
                 l2 = 0.01,#正則化のλパラメータ
                 epochs = 50, #n_epochs, #トレーニング回数
                 eta = 0.0005, #学習率
                 minibatch_size = 1, 
                 shuffle = True, #各エポックでデータをシャッフルするかどうか
                 seed = 1)

In [2]:
import pandas as pd
import math
import numpy as np

######################################################
########## 入力値：ランダムウォークの設定 ############
######################################################

# ランダムウォークのデータ
df_sensor = pd.read_csv('2dimention_sensor_data_2.csv')
length = len(df_sensor)

X_sensor = np.array(df_sensor.loc[:,'front':'left'])
#X_sensor = X_sensor.astype(int)

action = np.array(np.zeros(length))
action = np.array(df_sensor.loc[:,'action'])

# 各行の最小センサ値の列名
minimum_kind = df_sensor.loc[:,'front':'left'].idxmin(axis = 1)
# 教師の初期化
#teacher = np.array(np.zeros((length,4)))
teacher = np.zeros((length,4))

for i in range(1, length):
    
    min_fir_before = sorted(X_sensor[i-1])[0]
    min_sec_before = sorted(X_sensor[i-1])[1]
    #print(min_fir_before)
    senor_min_before = math.sqrt(min_fir_before**2 + min_sec_before**2)
    
    min_fir_now = sorted(X_sensor[i])[0]
    min_sec_now = sorted(X_sensor[i])[1]
    
    senor_min_now = math.sqrt(min_fir_now**2 + min_sec_now**2)
    
    if action[i] == -1:
        action[i] = 0
    elif action[i] == 1:
        action[i] = 1
    elif action[i] == 3:
        action[i] = 2
    elif action[i] == -3:
        action[i] = 3
    
    # 前に動いたとき
    if action[i] == 1:
        # 正解
        if senor_min_now - senor_min_before > 0: 
            teacher[i] = (1,0,0,0)
        # 不正解
        else: 
            teacher[i] = (0,0.5,0.5,0.5)
            
    # 後に動いたとき
    if action[i] == 0:
        # 正解
        if senor_min_now - senor_min_before > 0: 
            teacher[i] = (0,1,0,0)
        # 不正解
        else: 
            teacher[i] = (0.5,0,0.5,0.5)
    
    # 右に進んだとき
    elif action[i] == 2:
        if senor_min_now - senor_min_before > 0:
            teacher[i] = (0,0,1,0)
        # 不正解
        else:
            teacher[i] = (0.5,0.5,0,0.5)
  
    # 左に進んだとき
    elif action[i] == 3:
        # 正解
        if senor_min_now - senor_min_before > 0:
            teacher[i] = (0,0,0,1)
        # 不正解
        else: 
            teacher[i] = (0.5,0.5,0.5,0)

X_sensor = np.delete(X_sensor, 0,0)
teacher = np.delete(teacher, 0,0)
teacher = teacher.astype(int)
action = np.delete(action, 0,0)

print(action)
#print(teacher)

[0 0 0 3 0 2 1 3 2 0 1 3 2 1 0 2 0 2 0 2 3 0 3 0 0 3 2 1 0 0 0 3 3 3 1 3 3
 1 2 0 0 1 1 3 3 1 2 0 3 0 3 1 2 2 2 0 3 0 1 3 2 1 0 2 0 2 0]


In [ ]:
################## 学習 ######################
for i in range(1):
    
    fit = nn.fit(X_train = X_sensor, 
                 
                 y_train_enc = teacher)

#################### 予測 ####################
pre = nn.predict(X_sensor)
print(pre)

In [3]:
np.zeros(5).shape

(5,)